In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import dolfin

In [2]:
import numpy as np
x = np.array([[1,2]]).T
A = np.array([[1,8], [3,4]])

In [3]:
coefs = [
    (0,0),
    (1,0),
    (0,1),
    (1,1),
    (2,0),
    (0,2)
]

In [4]:
#c = 'exp(1+cos(2*pi*x[0])/4.+cos(2*pi*x[1])/9.+cos(2*pi*(x[0]+x[1]))/16.+cos(4*pi*x[0])/25.+cos(4*pi*x[1])/36.)'
c = 'exp(1+cos(2*pi*x[0])+cos(2*pi*x[1])+cos(2*pi*(x[0]+x[1]))+cos(4*pi*x[0])+cos(4*pi*x[1]))'
#c = ' + '.join(["cos(2*pi*({}*x[0]+{}*x[1])) / {}".format(coefs[k][0], coefs[k][1], (k+1.)**2) for k in xrange(6)])
#c = 'exp({})'.format(c)
c_expr = dolfin.Expression(c)

mesh_size = 1000
mesh = dolfin.UnitSquareMesh(mesh_size, mesh_size)
fn_space = dolfin.FunctionSpace(mesh, 'Lagrange', 1)

v = dolfin.TestFunction(fn_space)
u = dolfin.TrialFunction(fn_space)

a = -dolfin.inner(dolfin.nabla_grad(v), c_expr*dolfin.nabla_grad(u))
f = dolfin.inner(v, dolfin.Constant(0))

u_sol = dolfin.Function(fn_space)

def x2_on_left_bdy(x, on_bdy):
    return on_bdy and dolfin.near(x[1], 0.)

def x2_on_right_bdy(x, on_bdy):
    return on_bdy and dolfin.near(x[1], 1.)

bcs = [
    dolfin.DirichletBC(fn_space, dolfin.Expression('x[0]'), x2_on_left_bdy),
    dolfin.DirichletBC(fn_space, dolfin.Expression('1-x[0]'), x2_on_right_bdy)
]

# neumann conditions automatically satisfied since they are homogeneous
dolfin.solve(a*dolfin.dx == f*dolfin.dx, u_sol, bcs)

DEBUG:FFC:Reusing form from cache.
DEBUG:FFC:Reusing form from cache.
DEBUG:FFC:Reusing form from cache.


In [5]:
# for ipython notebook
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.tri as tri

def mesh2triang(mesh):
    xy = mesh.coordinates()
    return tri.Triangulation(xy[:, 0], xy[:, 1], mesh.cells())

def plot(obj):
    plt.gca().set_aspect('equal')
    if isinstance(obj, dolfin.Function):
        mesh = obj.function_space().mesh()
        if (mesh.geometry().dim() != 2):
            raise(AttributeError)
        if obj.vector().size() == mesh.num_cells():
            C = obj.vector().array()
            plt.tripcolor(mesh2triang(mesh), C)
        else:
            C = obj.compute_vertex_values(mesh)
            plt.tripcolor(mesh2triang(mesh), C, shading='gouraud')
    elif isinstance(obj, dolfin.Mesh):
        if (obj.geometry().dim() != 2):
            raise(AttributeError)
        plt.triplot(mesh2triang(obj), color='k')


In [6]:
plt.figure(figsize=(8,8))
plot(u_sol)
plt.colorbar()
plt.gcf().patch.set_alpha(0.0)
plt.axis('scaled')
plt.savefig('model_kl.pdf', bbox_inches='tight')

KeyboardInterrupt: 

ValueError: Format "png" is not supported.
Supported formats: pdf.

In [7]:
import numpy as np
X,Y = np.mgrid[0:1:6j, 0:1:6j]
x,y = X.ravel(), Y.ravel()

In [8]:
sol_points = np.vectorize(u_sol)(x,y)

In [9]:
np.vectorize(u_sol)(np.linspace(0,1,51),np.linspace(0,1,51))

array([  3.53897772e-18,   3.11101647e-02,   4.58300971e-02,
         5.58490461e-02,   6.38656870e-02,   7.13102301e-02,
         7.93493294e-02,   8.93783454e-02,   1.03440162e-01,
         1.24562572e-01,   1.56666756e-01,   2.03426138e-01,
         2.65840211e-01,   3.39792152e-01,   4.16157651e-01,
         4.84533792e-01,   5.37993871e-01,   5.75129877e-01,
         5.98626160e-01,   6.12561650e-01,   6.20529914e-01,
         6.25021036e-01,   6.27540601e-01,   6.28927656e-01,
         6.29622545e-01,   6.29833802e-01,   6.29622545e-01,
         6.28927656e-01,   6.27540601e-01,   6.25021036e-01,
         6.20529914e-01,   6.12561650e-01,   5.98626160e-01,
         5.75129877e-01,   5.37993871e-01,   4.84533792e-01,
         4.16157651e-01,   3.39792152e-01,   2.65840211e-01,
         2.03426138e-01,   1.56666756e-01,   1.24562572e-01,
         1.03440162e-01,   8.93783454e-02,   7.93493294e-02,
         7.13102301e-02,   6.38656870e-02,   5.58490461e-02,
         4.58300971e-02,

In [10]:
np.savetxt('true_linear_elliptic_nodenom.txt', np.c_[x,y,sol_points], header='x,y,u')